# 가설
- 아이템의 카테고리별로 월별 판매량이 다를것이다
- 가게별로 아이템의 평균 가격이 다를것이다.가격에 따라 판매량에 차이가 있을것이다.
- 온라인, 오프라인에 따라 판매량의 차이가 있을것이다.
- 피쳐의 수를 간추리면 성능이 높아질 것이다.

# Import data

In [683]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from googletrans import Translator 

In [684]:
df_test = ("../files/test.csv")
df_train = ("../files/sales_train_v2.csv")

df_test = pd.read_csv(df_test, sep = ",") #(214200, 3)_
df_train = pd.read_csv(df_train, sep = ",") # (2935849, 6)
df_train['index'] = df_train.index
df_test['index'] = df_test.index

In [685]:
df_items = ("../files/items.csv")
df_item_categories = ("../files/item_categories.csv")
df_shops = ("../files/shops.csv")

df_items = pd.read_csv(df_items, sep = ",") #(214200, 3)
df_item_categories = pd.read_csv(df_item_categories, sep = ",") # (2935849, 6) 
df_shops = pd.read_csv(df_shops, sep = ",") #(214200, 3)

In [686]:
df_train.isnull().sum() #no null
df_test.isnull().sum() #no null

ID         0
shop_id    0
item_id    0
index      0
dtype: int64

In [687]:
df_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,index
0,02.01.2013,0,59,22154,999.00,1.0,0
1,03.01.2013,0,25,2552,899.00,1.0,1
2,05.01.2013,0,25,2552,899.00,-1.0,2
3,06.01.2013,0,25,2554,1709.05,1.0,3
4,15.01.2013,0,25,2555,1099.00,1.0,4


In [688]:
df_test.head()

,ID,shop_id,item_id,index
0,0,5,5037,0
1,1,5,5320,1
2,2,5,5233,2
3,3,5,5232,3
4,4,5,5268,4


In [689]:
df_item_categories.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [690]:
df_items.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


# Translate

- 물건 카테고리이름, 상점이름등은 중요한 피쳐라 생각해 해석한 뒤 전처리하였다.


- 아이템명은 중복을 제외하고 약 20000개정도의 값을 가짐. 불필요한 정보라 생각해 드롭하였다. 


- 데이터프레임끼리 merge 하는것이 편집하기 편하기 때문에 아이템명은 드롭하지 않은 상태.



In [691]:
translator = Translator()
item_category_name_ko = pd.Series([str(translator.translate(category_name, dest = "ko").text) for category_name in df_item_categories.item_category_name])
df_item_categories.item_category_name = item_category_name_ko
df_item_categories.item_category_name


shop_name_ko = pd.Series([str(translator.translate(shop_nm, dest = "ko").text) for shop_nm in df_shops.shop_name])
df_shops.shop_name = shop_name_ko
df_shops.shop_name

0                        Yakutsk Ordzhonikidze, 56 프랑
1                                     야쿠 츠크 TC "중앙"프랑
2                                  아디 게아 쇼핑 센터 "Mega"
3                              발라 시카 TRK "10 월 키노 미르"
4            볼츠키 (Volzhsky) 쇼핑 센터 "볼가 몰 (Volga Mall)"
5                       볼 로그 다 쇼핑 및 엔터테인먼트 센터 "마멀레이드"
6                          보로 네즈 (Porkhanovskaya, 13)
7                                Voronezh TRC "막시 미르"
8                            Voronezh TRC 도시 공원 "졸업생"
9                                               외환 거래
10                          Zhukovsky Str. 차 칼로프 39m?
11                       Zhukovsky Str. Chkalov 39 m²
12                                       온라인 상점 비상 사태
13                                    카잔 TC "Bahetle"
14                                   카잔 TC "파크 하우스"II
15                                   칼루가 TRC "XXI 세기"
16                                          콜로세움 "리오"
17    크라스 노야 르 스크 (Krasnoyarsk) 쇼핑 센터 "Vzletka Plaza"
18             크라스 노야 르 스크 (

# Simplify Categories 
- shopid의 인덱스를 줄여여주기위해 지역별, 동부 서부 중부 기타로 인덱스를 변경

In [692]:
def split_words(df_col_values):
    tokenized = [i.split(" ") for i in df_col_values]
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+') 
    
    for i in range(len(df_col_values)):
        df_col_values[i] = tokenized[i][0] if hangul.sub('',tokenized[i][0]) else "기타"
    return df_col_values

df_item_category= pd.DataFrame({"item_category_id" : df_item_categories.item_category_name.index, "new_item_category_name": split_words(df_item_categories.item_category_name.values)})
# split_words(df_shops.shop_name.values)
df_item_category

,item_category_id,new_item_category_name
0,0,기타
1,1,액세서리
2,2,액세서리
3,3,액세서리
4,4,액세서리
5,5,액세서리
6,6,액세서리
7,7,액세서리
8,8,티켓
9,9,상품


#  add city info

In [693]:
df_shops['location'] = None
df_shops.loc[0:1,'location'] = '야쿠츠크'
df_shops.loc[20:33,'location'] = '모스크바'
df_shops.loc[2,'location'] = '아디게아'
df_shops.loc[3,'location'] ='발라시카' 
df_shops.loc[4,'location'] = '볼츠키'
df_shops.loc[5,'location'] = '볼로그다'
df_shops.loc[6:8,'location'] = '보로네시'
df_shops.loc[9,'location'] = '기타'
df_shops.loc[10:11,'location'] = '기타'
df_shops.loc[12,'location'] = '기타'
df_shops.loc[13:14,'location'] = '카잔'
df_shops.loc[15,'location'] = '칼루가'
df_shops.loc[16,'location'] = '기타'
df_shops.loc[17:18,'location'] = '크라스노야르스크'
df_shops.loc[34:35,'location'] = '노브고로드'
df_shops.loc[19,'location'] = '쿠르스크'
df_shops.loc[36:37,'location'] ='노보시비르스크'
df_shops.loc[38,'location']='옴스크'
df_shops.loc[39:41,'location']='로스토프나도누'
df_shops.loc[42:43,'location']='상트페테르부르크'
df_shops.loc[44:45,'location']='사마라'
df_shops.loc[46,'location']='세르기예프 포사트'
df_shops.loc[47,'location']='수르구트'
df_shops.loc[48,'location']='톰스크'
df_shops.loc[49:51,'location']='튜멘'
df_shops.loc[52:53,'location']='우파'
df_shops.loc[54,'location']='힘키'
df_shops.loc[55:56,'location']='기타'
df_shops.loc[57:58,'location']= '야쿠츠크'
df_shops.loc[59,'location']='야로슬라블'


# Add direction info

In [694]:
df_shops['direction'] = None
df_shops.loc[0:1,'direction'] = '동부'
df_shops.loc[20:33,'direction'] = '서부'
df_shops.loc[2,'direction'] = '서부'
df_shops.loc[3,'direction'] ='서부' 
df_shops.loc[4,'direction'] = '서부'
df_shops.loc[5,'direction'] = '서부'
df_shops.loc[6:8,'direction'] = '서부'
df_shops.loc[9,'direction'] = '기타'
df_shops.loc[10:11,'direction'] = '기타'
df_shops.loc[12,'direction'] = '기타'
df_shops.loc[13:14,'direction'] = '서부'
df_shops.loc[15,'direction'] = '서부'
df_shops.loc[16,'direction'] = '기타'
df_shops.loc[17:18,'direction'] = '중부'
df_shops.loc[34:35,'direction'] = '서부'
df_shops.loc[19,'direction'] = '서부'
df_shops.loc[36:37,'direction'] ='중부'
df_shops.loc[38,'direction']='중부'
df_shops.loc[39:41,'direction']='서부'
df_shops.loc[42:43,'direction']='서부'
df_shops.loc[44:45,'direction']='서부'
df_shops.loc[46,'direction']='서부'
df_shops.loc[47,'direction']='중부'
df_shops.loc[48,'direction']='중부'
df_shops.loc[49:51,'direction']='중부'
df_shops.loc[52:53,'direction']='서부'
df_shops.loc[54,'direction']='서부'
df_shops.loc[55:56,'direction']='기타'
df_shops.loc[57:58,'direction']= '동부'
df_shops.loc[59,'direction']='서부'
df_shops

,shop_name,shop_id,location,direction
0,"Yakutsk Ordzhonikidze, 56 프랑",0,야쿠츠크,동부
1,"야쿠 츠크 TC ""중앙""프랑",1,야쿠츠크,동부
2,"아디 게아 쇼핑 센터 ""Mega""",2,아디게아,서부
3,"발라 시카 TRK ""10 월 키노 미르""",3,발라시카,서부
4,"볼츠키 (Volzhsky) 쇼핑 센터 ""볼가 몰 (Volga Mall)""",4,볼츠키,서부
5,"볼 로그 다 쇼핑 및 엔터테인먼트 센터 ""마멀레이드""",5,볼로그다,서부
6,"보로 네즈 (Porkhanovskaya, 13)",6,보로네시,서부
7,"Voronezh TRC ""막시 미르""",7,보로네시,서부
8,"Voronezh TRC 도시 공원 ""졸업생""",8,보로네시,서부
9,외환 거래,9,기타,기타


# Merge & Drop

- 이제 정제된 데이터들만 갖고가기 위해 Merge, Drop 해 주었다.


In [695]:
#아이템카테고리 id만 df_train,df_test에 존재 -> items + item_categories -> df_train + item_info
df_item_info = pd.merge(df_item_category, df_items, on = "item_category_id")
df_item_info.head(5)

,item_category_id,new_item_category_name,item_name,item_id
0,0,기타,PC: Гарнитура HyperX Cloud Core игровая стерео...,5441
1,0,기타,Наушники PHILIPS SBC HC8680,16255
2,0,기타,Наушники RITMIX RH-120,16256
3,0,기타,Наушники RITMIX RH-124 Black,16257
4,1,액세서리,PS2: Карта памяти 8 Мб черная (Memory Card 8Mb...,5606


In [696]:
del df_item_info["item_name"] #item_name의 데이터는 너무 방대함 -> 카테고리로 충분하다 판단

In [697]:
df_item_info.head(5)

,item_category_id,new_item_category_name,item_id
0,0,기타,5441
1,0,기타,16255
2,0,기타,16256
3,0,기타,16257
4,1,액세서리,5606


In [698]:
df_train = pd.merge(df_train,df_item_info,on = "item_id")
df_test = pd.merge(df_test,df_item_info,on = "item_id")

In [699]:
df_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,index,item_category_id,new_item_category_name
0,02.01.2013,0,59,22154,999.0,1.0,0,37,시네마
1,23.01.2013,0,24,22154,999.0,1.0,3270,37,시네마
2,20.01.2013,0,27,22154,999.0,1.0,17081,37,시네마
3,02.01.2013,0,25,22154,999.0,1.0,25918,37,시네마
4,03.01.2013,0,25,22154,999.0,1.0,25919,37,시네마


In [701]:
df_test.head()

,ID,shop_id,item_id,index,item_category_id,new_item_category_name
0,0,5,5037,0,19,게임
1,5100,4,5037,5100,19,게임
2,10200,6,5037,10200,19,게임
3,15300,3,5037,15300,19,게임
4,20400,2,5037,20400,19,게임


In [702]:
df_train = pd.merge(df_train, df_shops, on = "shop_id")
df_test = pd.merge(df_test, df_shops, on = "shop_id")

In [703]:
df_test.head()

,ID,shop_id,item_id,index,item_category_id,new_item_category_name,shop_name,location,direction
0,0,5,5037,0,19,게임,"볼 로그 다 쇼핑 및 엔터테인먼트 센터 ""마멀레이드""",볼로그다,서부
1,1,5,5320,1,55,음악,"볼 로그 다 쇼핑 및 엔터테인먼트 센터 ""마멀레이드""",볼로그다,서부
2,2,5,5233,2,19,게임,"볼 로그 다 쇼핑 및 엔터테인먼트 센터 ""마멀레이드""",볼로그다,서부
3,3,5,5232,3,23,게임,"볼 로그 다 쇼핑 및 엔터테인먼트 센터 ""마멀레이드""",볼로그다,서부
4,4,5,5268,4,20,게임,"볼 로그 다 쇼핑 및 엔터테인먼트 센터 ""마멀레이드""",볼로그다,서부


In [704]:
df_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,index,item_category_id,new_item_category_name,shop_name,location,direction
0,02.01.2013,0,59,22154,999.0,1.0,0,37,시네마,"야로 슬라 블 쇼핑 센터 ""Altair""",야로슬라블,서부
1,16.05.2013,4,59,2573,249.0,1.0,498629,55,음악,"야로 슬라 블 쇼핑 센터 ""Altair""",야로슬라블,서부
2,26.01.2013,0,59,2574,399.0,1.0,77959,55,음악,"야로 슬라 블 쇼핑 센터 ""Altair""",야로슬라블,서부
3,09.01.2013,0,59,2574,399.0,1.0,77961,55,음악,"야로 슬라 블 쇼핑 센터 ""Altair""",야로슬라블,서부
4,24.02.2013,1,59,2574,399.0,1.0,133013,55,음악,"야로 슬라 블 쇼핑 센터 ""Altair""",야로슬라블,서부


# Direction To Shop Id

In [705]:
df_test["shop_id"] = df_test["direction"]
df_train["shop_id"] = df_test["direction"]
del df_test["direction"]
del df_train["direction"]

In [706]:
df_test

,ID,shop_id,item_id,index,item_category_id,new_item_category_name,shop_name,location
0,0,서부,5037,0,19,게임,"볼 로그 다 쇼핑 및 엔터테인먼트 센터 ""마멀레이드""",볼로그다
1,1,서부,5320,1,55,음악,"볼 로그 다 쇼핑 및 엔터테인먼트 센터 ""마멀레이드""",볼로그다
2,2,서부,5233,2,19,게임,"볼 로그 다 쇼핑 및 엔터테인먼트 센터 ""마멀레이드""",볼로그다
3,3,서부,5232,3,23,게임,"볼 로그 다 쇼핑 및 엔터테인먼트 센터 ""마멀레이드""",볼로그다
4,4,서부,5268,4,20,게임,"볼 로그 다 쇼핑 및 엔터테인먼트 센터 ""마멀레이드""",볼로그다
5,5,서부,5039,5,23,게임,"볼 로그 다 쇼핑 및 엔터테인먼트 센터 ""마멀레이드""",볼로그다
6,6,서부,5041,6,20,게임,"볼 로그 다 쇼핑 및 엔터테인먼트 센터 ""마멀레이드""",볼로그다
7,7,서부,5046,7,55,음악,"볼 로그 다 쇼핑 및 엔터테인먼트 센터 ""마멀레이드""",볼로그다
8,8,서부,5319,8,55,음악,"볼 로그 다 쇼핑 및 엔터테인먼트 센터 ""마멀레이드""",볼로그다
9,9,서부,5003,9,20,게임,"볼 로그 다 쇼핑 및 엔터테인먼트 센터 ""마멀레이드""",볼로그다


In [707]:
df_train

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,index,item_category_id,new_item_category_name,shop_name,location
0,02.01.2013,0,서부,22154,999.0,1.0,0,37,시네마,"야로 슬라 블 쇼핑 센터 ""Altair""",야로슬라블
1,16.05.2013,4,서부,2573,249.0,1.0,498629,55,음악,"야로 슬라 블 쇼핑 센터 ""Altair""",야로슬라블
2,26.01.2013,0,서부,2574,399.0,1.0,77959,55,음악,"야로 슬라 블 쇼핑 센터 ""Altair""",야로슬라블
3,09.01.2013,0,서부,2574,399.0,1.0,77961,55,음악,"야로 슬라 블 쇼핑 센터 ""Altair""",야로슬라블
4,24.02.2013,1,서부,2574,399.0,1.0,133013,55,음악,"야로 슬라 블 쇼핑 센터 ""Altair""",야로슬라블
5,29.03.2013,2,서부,2574,399.0,1.0,334238,55,음악,"야로 슬라 블 쇼핑 센터 ""Altair""",야로슬라블
6,19.07.2013,6,서부,2574,399.0,1.0,706298,55,음악,"야로 슬라 블 쇼핑 센터 ""Altair""",야로슬라블
7,03.07.2013,6,서부,2574,399.0,1.0,706299,55,음악,"야로 슬라 블 쇼핑 센터 ""Altair""",야로슬라블
8,06.07.2013,6,서부,2574,399.0,1.0,706307,55,음악,"야로 슬라 블 쇼핑 센터 ""Altair""",야로슬라블
9,28.08.2013,7,서부,2574,399.0,1.0,736350,55,음악,"야로 슬라 블 쇼핑 센터 ""Altair""",야로슬라블


# Price Rank Per Shops and Items

- 가격정보는 명목형 값이므로 최소화해야한다는 생각을 하였다.


- 한 아이템내의 가격을 sort 한 뒤 오름차순으로 등수를 매겼다.



- 딕트타입으로 row 의 인덱스값을 키값, 등수를 매긴 가격정보를 밸류값으로 저장한 뒤 데이터프레임 생성

In [708]:
df_grouped = df_train.groupby(["shop_id","item_category_id","item_id"])["item_price"].apply(list)
#같은 제품에 있는 가격들을 list로 묶음
ranked_price_mat = []
for price_list in df_grouped.values:
    ranked_price_dict = {}
    ranked_price = {price : no  for no,price in enumerate(sorted(np.unique(price_list)))}
    ranked_price_dict.update(ranked_price)
    ranked_price_mat.append(ranked_price_dict)
    
# 가격에 따라 enumerate로 등수를 매김, 현재 가격과 등수가 각각 dict값의 키와 밸류로 저장되어있음

for value_no in range(len(df_grouped.values)): 
    for index in range(len(df_grouped.values[value_no])):
        df_grouped.values[value_no][index] = ranked_price_mat[value_no][df_grouped.values[value_no][index]]
        
    #df_grouped에 변경사항 반영

df_get_back = df_train.groupby(["shop_id","item_category_id","item_id"])["index"].apply(list)
get_back_dict = {}
for value_no in range(len(df_grouped.values)): 
    for index in range(len(df_grouped.values[value_no])):
        get_back_dict.update({df_get_back.values[value_no][index] : df_grouped.values[value_no][index]+1})
        
    #item_price를 원래대로 돌리기 위해 index값을 똑같이 묶은 뒤 딕트타입으로 키값에 인덱스번호를 부여
    #그리고 enumerate한 등수가 0이되지 않도록 1을 더해줌. {인덱스값 : 가격} 형태의 딕트 완성
        
df_ranked = pd.DataFrame(list(get_back_dict.items()),columns=['index','new_item_price'])
df_ranked = pd.merge(df_train,df_ranked, on = "index")

In [709]:
df_ranked

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,index,item_category_id,new_item_category_name,shop_name,location,new_item_price
0,02.01.2013,0,서부,22154,999.0,1.0,0,37,시네마,"야로 슬라 블 쇼핑 센터 ""Altair""",야로슬라블,2
1,16.05.2013,4,서부,2573,249.0,1.0,498629,55,음악,"야로 슬라 블 쇼핑 센터 ""Altair""",야로슬라블,1
2,26.01.2013,0,서부,2574,399.0,1.0,77959,55,음악,"야로 슬라 블 쇼핑 센터 ""Altair""",야로슬라블,3
3,09.01.2013,0,서부,2574,399.0,1.0,77961,55,음악,"야로 슬라 블 쇼핑 센터 ""Altair""",야로슬라블,3
4,24.02.2013,1,서부,2574,399.0,1.0,133013,55,음악,"야로 슬라 블 쇼핑 센터 ""Altair""",야로슬라블,3
5,29.03.2013,2,서부,2574,399.0,1.0,334238,55,음악,"야로 슬라 블 쇼핑 센터 ""Altair""",야로슬라블,3
6,19.07.2013,6,서부,2574,399.0,1.0,706298,55,음악,"야로 슬라 블 쇼핑 센터 ""Altair""",야로슬라블,3
7,03.07.2013,6,서부,2574,399.0,1.0,706299,55,음악,"야로 슬라 블 쇼핑 센터 ""Altair""",야로슬라블,3
8,06.07.2013,6,서부,2574,399.0,1.0,706307,55,음악,"야로 슬라 블 쇼핑 센터 ""Altair""",야로슬라블,3
9,28.08.2013,7,서부,2574,399.0,1.0,736350,55,음악,"야로 슬라 블 쇼핑 센터 ""Altair""",야로슬라블,3


In [710]:
df_ranked.isnull().sum()

date                      0
date_block_num            0
shop_id                   0
item_id                   0
item_price                0
item_cnt_day              0
index                     0
item_category_id          0
new_item_category_name    0
shop_name                 0
location                  0
new_item_price            0
dtype: int64

In [711]:
df_ranked_light = df_ranked.drop(["date","date_block_num","item_id","item_price","item_cnt_day","index","item_category_id","shop_id"], axis = 1)

df_mean_price_rank_shop = df_ranked_light.groupby(["shop_name"], as_index = False)["new_item_price"].mean() #null 0
 
df_std_price_rank_shop = df_ranked_light.groupby(["shop_name"], as_index = True)["new_item_price"].std() #null 0
df_std_price_rank_shop = pd.DataFrame({"shop_name" : df_std_price_rank_shop.index, "new_item_price" : df_std_price_rank_shop.values})

df_mean_price_rank_item = df_ranked_light.groupby(["new_item_category_name"], as_index = False)["new_item_price"].mean().fillna(1) #null 2, mode 1
df_std_price_rank_item = df_ranked_light.groupby(["new_item_category_name"], as_index = True)["new_item_price"].std().fillna(0) #null 2 mode 0
df_std_price_rank_item = pd.DataFrame({"new_item_category_name" :df_std_price_rank_item.index, "df_std_price_rank_item" : df_std_price_rank_item.values})

In [657]:
# df_mean_price_rank_shop.isnull().sum()
# df_std_price_rank_shop.isnull().sum()
# df_mean_price_rank_item.isnull().sum()
# df_std_price_rank_item.isnull().sum()
# df_rowest_gap_price_item["new_item_price"].mode()

In [712]:
df_price_rank_per_shop = pd.merge(df_mean_price_rank_shop,df_std_price_rank_shop, on = "shop_name")
del df_mean_price_rank_shop
del df_std_price_rank_shop
df_price_rank_per_shop = df_price_rank_per_shop.rename(index = str, columns ={"shop_name":"shop_name","new_item_price_x" : "mean_price_rank_shop","new_item_price_y" : "mean_price_rank_shop"})

In [713]:
df_price_rank_per_item = pd.merge(df_std_price_rank_item, df_mean_price_rank_item, on = "new_item_category_name")
del df_std_price_rank_item
del df_mean_price_rank_item
df_price_rank_per_item = df_price_rank_per_item.rename(index = str, columns ={"df_std_price_rank_item":"df_std_price_rank_item","new_item_category_name" : "new_item_category_name","new_item_price" : "mean_price_rank_item"})

In [714]:
df_price_rank_per_item

,df_std_price_rank_item,new_item_category_name,mean_price_rank_item
0,5.476767,게임,5.530410
1,0.594879,공식,1.602740
2,5.976896,기타,6.507326
3,0.036974,도서,1.001368
4,109.079712,사무실,176.721750
5,1.223120,선물,1.945656
6,2.914580,시네마,2.738108
7,3.430089,액세서리,4.295131
8,0.890043,영화,1.591907
9,0.935440,음식,1.673423


In [715]:
df_train = pd.merge(df_price_rank_per_item, df_train, on = "new_item_category_name")
df_train = pd.merge(df_price_rank_per_shop, df_train, on = "shop_name")

In [716]:
df_test = pd.merge(df_price_rank_per_item, df_test, on = "new_item_category_name")
df_test= pd.merge(df_price_rank_per_shop, df_test, on = "shop_name")

In [717]:
df_train

,shop_name,mean_price_rank_shop,mean_price_rank_shop,df_std_price_rank_item,new_item_category_name,mean_price_rank_item,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,index,item_category_id,location
0,"모스크바 TC ""Budenovskiy""(pav.K7)",4.890274,15.816163,5.476767,게임,5.530410,16.02.2013,1,서부,2705,1259.00,1.0,153135,23,모스크바
1,"모스크바 TC ""Budenovskiy""(pav.K7)",4.890274,15.816163,5.476767,게임,5.530410,17.03.2013,2,서부,2705,1258.50,1.0,322170,23,모스크바
2,"모스크바 TC ""Budenovskiy""(pav.K7)",4.890274,15.816163,5.476767,게임,5.530410,12.05.2013,4,서부,2705,899.00,1.0,452583,23,모스크바
3,"모스크바 TC ""Budenovskiy""(pav.K7)",4.890274,15.816163,5.476767,게임,5.530410,29.07.2013,6,서부,2719,749.00,1.0,682153,19,모스크바
4,"모스크바 TC ""Budenovskiy""(pav.K7)",4.890274,15.816163,5.476767,게임,5.530410,22.12.2013,11,서부,2719,749.00,1.0,1136950,19,모스크바
5,"모스크바 TC ""Budenovskiy""(pav.K7)",4.890274,15.816163,5.476767,게임,5.530410,11.01.2014,12,서부,2719,749.00,1.0,1366496,19,모스크바
6,"모스크바 TC ""Budenovskiy""(pav.K7)",4.890274,15.816163,5.476767,게임,5.530410,17.08.2013,7,서부,2720,749.00,1.0,790772,23,모스크바
7,"모스크바 TC ""Budenovskiy""(pav.K7)",4.890274,15.816163,5.476767,게임,5.530410,24.01.2014,12,서부,2720,749.00,1.0,1366495,23,모스크바
8,"모스크바 TC ""Budenovskiy""(pav.K7)",4.890274,15.816163,5.476767,게임,5.530410,17.06.2013,5,서부,2748,799.00,1.0,596370,19,모스크바
9,"모스크바 TC ""Budenovskiy""(pav.K7)",4.890274,15.816163,5.476767,게임,5.530410,11.08.2013,7,서부,2748,799.00,1.0,790776,19,모스크바


In [718]:
df_test

,shop_name,mean_price_rank_shop,mean_price_rank_shop,df_std_price_rank_item,new_item_category_name,mean_price_rank_item,ID,shop_id,item_id,index,item_category_id,location
0,"모스크바 TC ""Budenovskiy""(pav.K7)",4.890274,15.816163,5.476767,게임,5.530410,66300,서부,5037,66300,19,모스크바
1,"모스크바 TC ""Budenovskiy""(pav.K7)",4.890274,15.816163,5.476767,게임,5.530410,66302,서부,5233,66302,19,모스크바
2,"모스크바 TC ""Budenovskiy""(pav.K7)",4.890274,15.816163,5.476767,게임,5.530410,66303,서부,5232,66303,23,모스크바
3,"모스크바 TC ""Budenovskiy""(pav.K7)",4.890274,15.816163,5.476767,게임,5.530410,66304,서부,5268,66304,20,모스크바
4,"모스크바 TC ""Budenovskiy""(pav.K7)",4.890274,15.816163,5.476767,게임,5.530410,66305,서부,5039,66305,23,모스크바
5,"모스크바 TC ""Budenovskiy""(pav.K7)",4.890274,15.816163,5.476767,게임,5.530410,66306,서부,5041,66306,20,모스크바
6,"모스크바 TC ""Budenovskiy""(pav.K7)",4.890274,15.816163,5.476767,게임,5.530410,66309,서부,5003,66309,20,모스크바
7,"모스크바 TC ""Budenovskiy""(pav.K7)",4.890274,15.816163,5.476767,게임,5.530410,66311,서부,4843,66311,20,모스크바
8,"모스크바 TC ""Budenovskiy""(pav.K7)",4.890274,15.816163,5.476767,게임,5.530410,66313,서부,4869,66313,22,모스크바
9,"모스크바 TC ""Budenovskiy""(pav.K7)",4.890274,15.816163,5.476767,게임,5.530410,66314,서부,4870,66314,23,모스크바


In [719]:
del df_train["shop_name"]
del df_test["shop_name"]

del df_test["index"]
del df_train["index"]

del df_train["item_category_id"]
del df_train["item_price"]

In [720]:
df_train["item_id"] = df_train["new_item_category_name"]
del df_train["new_item_category_name"]

In [721]:
df_train

,mean_price_rank_shop,mean_price_rank_shop,df_std_price_rank_item,mean_price_rank_item,date,date_block_num,shop_id,item_id,item_cnt_day,location
0,4.890274,15.816163,5.476767,5.530410,16.02.2013,1,서부,게임,1.0,모스크바
1,4.890274,15.816163,5.476767,5.530410,17.03.2013,2,서부,게임,1.0,모스크바
2,4.890274,15.816163,5.476767,5.530410,12.05.2013,4,서부,게임,1.0,모스크바
3,4.890274,15.816163,5.476767,5.530410,29.07.2013,6,서부,게임,1.0,모스크바
4,4.890274,15.816163,5.476767,5.530410,22.12.2013,11,서부,게임,1.0,모스크바
5,4.890274,15.816163,5.476767,5.530410,11.01.2014,12,서부,게임,1.0,모스크바
6,4.890274,15.816163,5.476767,5.530410,17.08.2013,7,서부,게임,1.0,모스크바
7,4.890274,15.816163,5.476767,5.530410,24.01.2014,12,서부,게임,1.0,모스크바
8,4.890274,15.816163,5.476767,5.530410,17.06.2013,5,서부,게임,1.0,모스크바
9,4.890274,15.816163,5.476767,5.530410,11.08.2013,7,서부,게임,1.0,모스크바


In [725]:
# df_test["item_id"] = df_test["new_item_category_name"]
# del df_test["new_item_category_name"]
# del df_test["item_category_id"]
# del df_test["item_cnt_day"]
# del df_test["item_category_id"]
# del df_test["item_price"]
df_test

,mean_price_rank_shop,mean_price_rank_shop,df_std_price_rank_item,mean_price_rank_item,ID,shop_id,item_id,location
0,4.890274,15.816163,5.476767,5.530410,66300,서부,게임,모스크바
1,4.890274,15.816163,5.476767,5.530410,66302,서부,게임,모스크바
2,4.890274,15.816163,5.476767,5.530410,66303,서부,게임,모스크바
3,4.890274,15.816163,5.476767,5.530410,66304,서부,게임,모스크바
4,4.890274,15.816163,5.476767,5.530410,66305,서부,게임,모스크바
5,4.890274,15.816163,5.476767,5.530410,66306,서부,게임,모스크바
6,4.890274,15.816163,5.476767,5.530410,66309,서부,게임,모스크바
7,4.890274,15.816163,5.476767,5.530410,66311,서부,게임,모스크바
8,4.890274,15.816163,5.476767,5.530410,66313,서부,게임,모스크바
9,4.890274,15.816163,5.476767,5.530410,66314,서부,게임,모스크바


In [732]:
del df_train["date"]
del df_train["date_block_num"]

In [735]:
df_test_2 = df_test.iloc[:,:4]
df_test_1 = df_test.iloc[:,4:]
df_test = pd.concat([df_test_1,df_test_2], axis = 1)
df_test

,ID,shop_id,item_id,location,mean_price_rank_shop,mean_price_rank_shop,df_std_price_rank_item,mean_price_rank_item
0,66300,서부,게임,모스크바,4.890274,15.816163,5.476767,5.530410
1,66302,서부,게임,모스크바,4.890274,15.816163,5.476767,5.530410
2,66303,서부,게임,모스크바,4.890274,15.816163,5.476767,5.530410
3,66304,서부,게임,모스크바,4.890274,15.816163,5.476767,5.530410
4,66305,서부,게임,모스크바,4.890274,15.816163,5.476767,5.530410
5,66306,서부,게임,모스크바,4.890274,15.816163,5.476767,5.530410
6,66309,서부,게임,모스크바,4.890274,15.816163,5.476767,5.530410
7,66311,서부,게임,모스크바,4.890274,15.816163,5.476767,5.530410
8,66313,서부,게임,모스크바,4.890274,15.816163,5.476767,5.530410
9,66314,서부,게임,모스크바,4.890274,15.816163,5.476767,5.530410


In [ ]:
len(df_train.columns)#7
len(df_test.columns) #8
df_train.columns

# Load Module & Dataset

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import calendar
from calendar import Calendar


%matplotlib inline

train_df = pd.read_csv("./sales_train_v2.csv") # sales_train_v2.csv 데일리한 매출 정보

In [ ]:
def Weekday_Columns_maker(Target_df):
    # 분할하여 추가
    date_df = pd.DataFrame(Target_df["date"].str.split(".").tolist(),  columns = ['Day','Month','Year'])
    # Date_df + Train_df = Full_df
    Full_df = pd.concat([train_df, date_df],axis= 1)


    # Function
    make_weekday_function = lambda x: datetime.weekday(datetime.strptime(x,  "%d.%m.%Y"))
    # Activate
    Full_df['weekday'] = pd.DataFrame(list(map(make_weekday_function,Full_df['date'])))

    # Sort (make 'weekday_str')
    weekday_sort_dict = {0:'Mon',1:'Tue',2:'Wed',3:'Thu',4:'Fri',5:'Sat',6:'Sun'}
    Full_df["weekday_str"] = Full_df.weekday.map(weekday_sort_dict)
    return Full_df
    

In [ ]:
def Date_Table_maker(Target_df):
    
    # 분할하여 추가
    date_df = pd.DataFrame(Target_df["date"].str.split(".").tolist(),  columns = ['Day','Month','Year'])
    # Date_df + Train_df = Full_df
    Full_df = pd.concat([train_df, date_df],axis= 1)


    # Function
    make_weekday_function = lambda x: datetime.weekday(datetime.strptime(x,  "%d.%m.%Y"))
    # Activate
    Full_df['weekday'] = pd.DataFrame(list(map(make_weekday_function,Full_df['date'])))

    # Sort (make 'weekday_str')
    weekday_sort_dict = {0:'Mon',1:'Tue',2:'Wed',3:'Thu',4:'Fri',5:'Sat',6:'Sun'}
    Full_df["weekday_str"] = Full_df.weekday.map(weekday_sort_dict)


    Just_Year = Full_df['Year'].unique()
    Just_Month = Full_df['Month'].unique()

    p=0
    MAKE=[]
    for Yidx,i in enumerate(Just_Year):
        for Midx,j in enumerate(Just_Month):
            if Midx == 11 and Yidx == 2:
                A = np.busday_count(i+'-'+j, '2016'+'-'+'01',weekmask=[1,0,0,0,0,0,0])
                B = np.busday_count(i+'-'+j, '2016'+'-'+'01',weekmask=[0,1,0,0,0,0,0]) 
                C = np.busday_count(i+'-'+j, '2016'+'-'+'01',weekmask=[0,0,1,0,0,0,0]) 
                D = np.busday_count(i+'-'+j, '2016'+'-'+'01',weekmask=[0,0,0,1,0,0,0]) 
                E = np.busday_count(i+'-'+j, '2016'+'-'+'01',weekmask=[0,0,0,0,1,0,0]) 
                F = np.busday_count(i+'-'+j, '2016'+'-'+'01',weekmask=[0,0,0,0,0,1,0]) 
                G = np.busday_count(i+'-'+j, '2016'+'-'+'01',weekmask=[0,0,0,0,0,0,1]) 
                SSEM = len(np.arange(i+'-'+j, '2016'+'-'+'01', dtype='datetime64[D]'))
                H= Just_Year[Yidx]
                I= Just_Month[Midx]
                p = p+1
                print(p,Yidx,Midx)
                MAKE.append([H,I,A,B,C,D,E,F,G,SSEM])
            elif Midx != 11:
                A = np.busday_count(i+'-'+j, i+'-'+Just_Month[Midx+1],weekmask=[1,0,0,0,0,0,0]) #“Mon Tue Wed Thu Fri"
                B = np.busday_count(i+'-'+j, i+'-'+Just_Month[Midx+1],weekmask=[0,1,0,0,0,0,0]) 
                C = np.busday_count(i+'-'+j, i+'-'+Just_Month[Midx+1],weekmask=[0,0,1,0,0,0,0]) 
                D = np.busday_count(i+'-'+j, i+'-'+Just_Month[Midx+1],weekmask=[0,0,0,1,0,0,0]) 
                E = np.busday_count(i+'-'+j, i+'-'+Just_Month[Midx+1],weekmask=[0,0,0,0,1,0,0]) 
                F = np.busday_count(i+'-'+j, i+'-'+Just_Month[Midx+1],weekmask=[0,0,0,0,0,1,0]) 
                G = np.busday_count(i+'-'+j, i+'-'+Just_Month[Midx+1],weekmask=[0,0,0,0,0,0,1]) 
                SSEM = len(np.arange(i+'-'+j, i+'-'+Just_Month[Midx+1], dtype='datetime64[D]'))
                H= Just_Year[Yidx]
                I= Just_Month[Midx]
                p = p+1
                print(p,Yidx,Midx)
                MAKE.append([H,I,A,B,C,D,E,F,G,SSEM])
            else :
                A = np.busday_count(i+'-'+j, Just_Year[Yidx+1]+'-'+'01',weekmask=[1,0,0,0,0,0,0])
                B = np.busday_count(i+'-'+j, Just_Year[Yidx+1]+'-'+'01',weekmask=[0,1,0,0,0,0,0]) 
                C = np.busday_count(i+'-'+j, Just_Year[Yidx+1]+'-'+'01',weekmask=[0,0,1,0,0,0,0]) 
                D = np.busday_count(i+'-'+j, Just_Year[Yidx+1]+'-'+'01',weekmask=[0,0,0,1,0,0,0]) 
                E = np.busday_count(i+'-'+j, Just_Year[Yidx+1]+'-'+'01',weekmask=[0,0,0,0,1,0,0]) 
                F = np.busday_count(i+'-'+j, Just_Year[Yidx+1]+'-'+'01',weekmask=[0,0,0,0,0,1,0]) 
                G = np.busday_count(i+'-'+j, Just_Year[Yidx+1]+'-'+'01',weekmask=[0,0,0,0,0,0,1]) 
                SSEM = len(np.arange(i+'-'+j, Just_Year[Yidx+1]+'-'+'01', dtype='datetime64[D]'))
                H= Just_Year[Yidx]
                I= Just_Month[Midx]
                p = p+1
                print(p,Yidx,Midx)
                MAKE.append([H,I,A,B,C,D,E,F,G,SSEM])

    Target_Calendar = pd.DataFrame(MAKE, columns=['Year','Month','Mon_count','Tue_count','Wed_count','Thu_count','Fri_count','Sat_count','Sun_count','Day_Count'])
    return Target_Calendar